In [31]:
pip install pykrige

In [32]:
from pykrige.uk import UniversalKriging
import pandas as pd
import numpy as np
from google.colab import files
import traceback

In [33]:
station_df=pd.read_excel('https://drive.google.com/uc?id=1DiAL0JzQQgSrUwXneznbe8OgDfqKKuKH', header=0)

In [34]:
station_df.head()

,Station_Number,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,1006,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.5100,128.1503,3.8
1,1007,TROUGHTON ISLAND,09/1956,1956-09-01,2021-01-31,-13.7542,126.1485,6.0
2,1009,KURI BAY,08/1961,1961-08-01,2012-09-01,-15.4875,124.5222,12.0
3,1013,WYNDHAM,01/1968,1968-01-01,2021-01-31,-15.4869,128.1236,11.0
4,1018,MOUNT ELIZABETH,01/1973,1973-01-01,2021-01-31,-16.4181,126.1025,546.0


In [35]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Station_Number  186 non-null    int64         
 1   Station_Name    186 non-null    object        
 2   Actual_open     186 non-null    object        
 3   Open            186 non-null    datetime64[ns]
 4   Close           186 non-null    datetime64[ns]
 5   LAT             186 non-null    float64       
 6   LON             186 non-null    float64       
 7   Height_m        186 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 11.8+ KB


In [36]:
station_df=station_df.rename(columns={"Station_Number": "station_number"})

In [37]:
station_df.head()

,station_number,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,1006,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.5100,128.1503,3.8
1,1007,TROUGHTON ISLAND,09/1956,1956-09-01,2021-01-31,-13.7542,126.1485,6.0
2,1009,KURI BAY,08/1961,1961-08-01,2012-09-01,-15.4875,124.5222,12.0
3,1013,WYNDHAM,01/1968,1968-01-01,2021-01-31,-15.4869,128.1236,11.0
4,1018,MOUNT ELIZABETH,01/1973,1973-01-01,2021-01-31,-16.4181,126.1025,546.0


In [38]:
data_df=pd.read_csv('http://calwildfire.thesofttools.com/WAWeatherStationData.csv', header=0)

In [39]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095213 entries, 0 to 1095212
Data columns (total 9 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   station_number                1095213 non-null  int64 
 1   date                          1095213 non-null  object
 2   station_mean_temperature_c    1095213 non-null  object
 3   station_air_temperature_c     1095213 non-null  object
 4   Station_TDMEAN_c              1095213 non-null  object
 5   Maximum_Sustained_Wind_Speed  1095213 non-null  object
 6   Mean_Wind_Speed               1095213 non-null  object
 7   Mean_Sea_Level_Pressure       1095213 non-null  object
 8   Mean_Station_Pressure         1095213 non-null  object
dtypes: int64(1), object(8)
memory usage: 75.2+ MB


In [40]:
#convert all string vaoues to float
data_df['date'] = pd.to_datetime(data_df['date'], format='%Y/%m/%d')
data_df['station_mean_temperature_c'] = pd.to_numeric(data_df['station_mean_temperature_c'], errors='coerce')
data_df['station_air_temperature_c'] = pd.to_numeric(data_df['station_air_temperature_c'], errors='coerce')
data_df['Station_TDMEAN_c'] = pd.to_numeric(data_df['Station_TDMEAN_c'], errors='coerce')
data_df['Maximum_Sustained_Wind_Speed'] = pd.to_numeric(data_df['Maximum_Sustained_Wind_Speed'], errors='coerce')
data_df['Mean_Wind_Speed'] = pd.to_numeric(data_df['Mean_Wind_Speed'], errors='coerce')
data_df['Mean_Sea_Level_Pressure'] = pd.to_numeric(data_df['Mean_Sea_Level_Pressure'], errors='coerce')
data_df['Mean_Station_Pressure'] = pd.to_numeric(data_df['Mean_Station_Pressure'], errors='coerce')

In [41]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095213 entries, 0 to 1095212
Data columns (total 9 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   station_number                1095213 non-null  int64         
 1   date                          1095213 non-null  datetime64[ns]
 2   station_mean_temperature_c    994114 non-null   float64       
 3   station_air_temperature_c     1049920 non-null  float64       
 4   Station_TDMEAN_c              1003964 non-null  float64       
 5   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 6   Mean_Wind_Speed               1032362 non-null  float64       
 7   Mean_Sea_Level_Pressure       791206 non-null   float64       
 8   Mean_Station_Pressure         793886 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 75.2 MB


In [42]:
data_df=data_df[['date','station_number', 'Station_TDMEAN_c', 'Mean_Wind_Speed',	'Mean_Sea_Level_Pressure', 'Mean_Station_Pressure', 'Maximum_Sustained_Wind_Speed']]

In [43]:
data_df.head()

,date,station_number,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed
0,2001-01-01,1006,20.6,13.7,1007.5,1007.0,NaN
1,2001-01-02,1006,20.7,9.5,1008.3,1007.8,NaN
2,2001-01-03,1006,22.4,12.7,1007.8,1007.3,NaN
3,2001-01-04,1006,22.7,10.9,1006.8,1006.3,NaN
4,2001-01-05,1006,23.9,14.6,1007.0,1006.5,NaN


In [44]:
station_data_df=pd.merge(data_df, station_df, how="left", on=["station_number"])

In [45]:
station_data_df.head()

,date,station_number,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,2001-01-01,1006,20.6,13.7,1007.5,1007.0,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
1,2001-01-02,1006,20.7,9.5,1008.3,1007.8,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
2,2001-01-03,1006,22.4,12.7,1007.8,1007.3,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
3,2001-01-04,1006,22.7,10.9,1006.8,1006.3,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
4,2001-01-05,1006,23.9,14.6,1007.0,1006.5,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8


In [46]:
station_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095213 entries, 0 to 1095212
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date                          1095213 non-null  datetime64[ns]
 1   station_number                1095213 non-null  int64         
 2   Station_TDMEAN_c              1003964 non-null  float64       
 3   Mean_Wind_Speed               1032362 non-null  float64       
 4   Mean_Sea_Level_Pressure       791206 non-null   float64       
 5   Mean_Station_Pressure         793886 non-null   float64       
 6   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 7   Station_Name                  1095213 non-null  object        
 8   Actual_open                   1095213 non-null  object        
 9   Open                          1095213 non-null  datetime64[ns]
 10  Close                         1095213 non-null  datetime64[ns]
 11

In [47]:
ver6_df=pd.read_excel('https://drive.google.com/uc?id=1P1W68_Q-JnpW9LqIb_63-uZBM4RdZkin', header=0)

In [48]:
ver6_df.head()

,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_perime,LAT,LON,ACRES,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,LGA
0,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,9.653766,-31.3497,116.0366,850.9043,h27v12,-31.350992,116.036200,36.224998,18.932627,1.0,206,148.566183,-31.35,116.05,0.0,29.32,13.40,15.27,15.10,1272.918883,2948,Chittering (S)
1,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,1.058290,-32.1413,116.1956,18.3947,h27v12,-32.142631,116.195939,61.924999,16.209669,1.0,266,151.398109,-32.15,116.20,0.0,29.39,14.58,14.62,14.69,1052.361097,52391,Armadale (C)
2,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.157169,-32.0572,116.1932,0.4853,h27v12,-32.059300,116.193215,66.547501,13.480727,1.0,312,233.564838,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,1025.507101,48743,Kalamunda (C)
3,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.558105,-32.0542,116.1940,5.3271,h27v12,-32.055134,116.192840,61.877499,13.119061,1.0,311,150.803304,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,733.392420,48743,Kalamunda (C)
4,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.165064,-32.0476,116.1925,0.5353,h27v12,-32.046801,116.192091,83.957497,14.985291,1.0,309,96.865831,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,755.563084,48743,Kalamunda (C)


In [49]:
#ver6_df['station_number']=np.nan
ver6_df['Station_TDMEAN_c']=np.nan
ver6_df['Mean_Wind_Speed']=np.nan
ver6_df['Mean_Sea_Level_Pressure']=np.nan
ver6_df['Mean_Station_Pressure']=np.nan
ver6_df['Maximum_Sustained_Wind_Speed']=np.nan
#ver6_df['delta']=np.nan

In [50]:
ver6_df.shape

(34932, 43)

In [51]:
uniq_dates=ver6_df['fih_date1'].drop_duplicates().reset_index().sort_values(by='fih_date1', ignore_index=True)
uniq_dates=uniq_dates.rename(columns={"fih_date1": "date"})

In [52]:
uniq_dates.head(1)

,index,date
0,0,2001-01-01


In [53]:
station_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095213 entries, 0 to 1095212
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date                          1095213 non-null  datetime64[ns]
 1   station_number                1095213 non-null  int64         
 2   Station_TDMEAN_c              1003964 non-null  float64       
 3   Mean_Wind_Speed               1032362 non-null  float64       
 4   Mean_Sea_Level_Pressure       791206 non-null   float64       
 5   Mean_Station_Pressure         793886 non-null   float64       
 6   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 7   Station_Name                  1095213 non-null  object        
 8   Actual_open                   1095213 non-null  object        
 9   Open                          1095213 non-null  datetime64[ns]
 10  Close                         1095213 non-null  datetime64[ns]
 11

In [54]:
temp_df=station_data_df[(station_data_df['date']==uniq_dates['date'][29]) & (station_data_df['Station_TDMEAN_c'].notnull())].reset_index(drop=True)
print(temp_df.to_string())

          date  station_number  Station_TDMEAN_c  Mean_Wind_Speed  Mean_Sea_Level_Pressure  Mean_Station_Pressure  Maximum_Sustained_Wind_Speed                              Station_Name Actual_open       Open      Close      LAT       LON  Height_m
0   2001-12-01            1006              23.6              6.7                   1004.6                 1004.1                           NaN  WYNDHAM AERO                                 01/1951 1951-01-01 2021-01-31 -15.5100  128.1503       3.8
1   2001-12-01            1007              24.8             12.8                   1005.7                 1004.8                           NaN  TROUGHTON ISLAND                             09/1956 1956-09-01 2021-01-31 -13.7542  126.1485       6.0
2   2001-12-01            1009              24.3             10.6                   1005.1                 1003.2                           NaN  KURI BAY                                     08/1961 1961-08-01 2012-09-01 -15.4875  124.5222      12.0
3   

In [55]:
ver6_df[ver6_df['fih_date1']==uniq_dates['date'][29]]

,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_perime,LAT,LON,ACRES,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,LGA,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed
1901,2001/2002,2001,SP,SWC,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,5.351121,-31.3861,115.5635,84.3533,h27v12,-31.384324,115.564899,59.292500,19.027012,1.0,49,238.001464,-31.40,115.55,0.0,20.91,12.96,11.39,12.00,2007.703180,4274,Gingin (S),NaN,NaN,NaN,NaN,NaN
1902,2001/2002,2001,SP,SWC,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,5.101086,-31.3778,115.5642,122.0496,h27v12,-31.375991,115.564409,49.972500,19.347919,1.0,45,202.133887,-31.40,115.55,0.0,20.91,12.96,11.39,12.00,2812.552783,4274,Gingin (S),NaN,NaN,NaN,NaN,NaN
1903,2001/2002,2001,SP,GLD,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,28.586300,-28.4186,118.9843,2801.4798,h28v11,-28.417761,118.986139,55.662498,13.100834,1.0,439,202.633700,-28.40,119.00,0.0,29.23,16.88,13.23,9.14,2575.926961,134,Sandstone (S),NaN,NaN,NaN,NaN,NaN
1904,2001/2002,2001,SP,SWC,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,4.100743,-31.3958,115.5608,36.6148,h27v12,-31.396824,115.560759,56.867500,18.453514,1.0,42,113.877683,-31.40,115.55,0.0,20.91,12.96,11.39,12.00,1126.432799,4274,Gingin (S),NaN,NaN,NaN,NaN,NaN
1905,2001/2002,2001,SP,PHL,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,17.228253,-31.3487,116.0330,1219.5767,h27v12,-31.346825,116.031060,12.392500,10.262343,1.0,203,278.185445,-31.35,116.05,0.0,21.03,11.64,10.95,10.60,1620.809995,2948,Chittering (S),NaN,NaN,NaN,NaN,NaN
1906,2001/2002,2001,SP,GLD,,999,999,2001-12-01,999,999,0,RS25,Cindy Miller,Actual Burnt,Fire History Model Project - Re-shaped polygon...,,11.017624,-28.4984,119.1692,1097.6776,h28v11,-28.496924,119.170094,55.662498,13.100834,1.0,440,185.888045,-28.50,119.15,0.0,30.36,17.19,13.34,9.35,1884.657165,134,Sandstone (S),NaN,NaN,NaN,NaN,NaN


In [56]:
#Test
UK = UniversalKriging(
      temp_df['LON'],
      temp_df['LAT'],
      temp_df['Station_TDMEAN_c'],
      variogram_model="spherical",
      verbose=False,
      enable_plotting=False,
      drift_terms=["regional_linear"],
)
temp2_df=ver6_df[ver6_df['fih_date1']==uniq_dates['date'][29]].reset_index(drop=True)
v,s2=UK.execute('points',temp2_df['LON'][0],temp2_df['LAT'][0])
v.data[0]

9.20478471726414

In [57]:
for i, day in uniq_dates.iterrows():
  if (i % 100)==0:
    print("Completed : "+str(round((i*100/uniq_dates.shape[0]),2))+"%")
  try:
    #Ordinary Kriging Station_TDMEAN_c
    #Here we use one day fo data as input for the kriging model and 5 parameter kriging
    #Station_TDMEAN_c
    #Get All station data related particular date
    input_df1=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Station_TDMEAN_c'].notnull())].reset_index(drop=True)
    UK_Station_TDMEAN_c = UniversalKriging(
          input_df1['LON'],
          input_df1['LAT'],
          input_df1['Station_TDMEAN_c'],
          variogram_model="spherical",
          verbose=False,
          enable_plotting=False,
          drift_terms=["regional_linear"],
    )
    #Mean_Wind_Speed
    input_df2=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Wind_Speed'].notnull())].reset_index(drop=True)
    UK_Mean_Wind_Speed = UniversalKriging(
          input_df2['LON'],
          input_df2['LAT'],
          input_df2['Mean_Wind_Speed'],
          variogram_model="spherical",
          verbose=False,
          enable_plotting=False,
          drift_terms=["regional_linear"],
    )
    #Mean_Sea_Level_Pressure
    input_df3=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Sea_Level_Pressure'].notnull())].reset_index(drop=True)
    UK_Mean_Sea_Level_Pressure = UniversalKriging(
          input_df3['LON'],
          input_df3['LAT'],
          input_df3['Mean_Sea_Level_Pressure'],
          variogram_model="spherical",
          verbose=False,
          enable_plotting=False,
          drift_terms=["regional_linear"],
    )
    #Mean_Station_Pressure.
    input_df4=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Station_Pressure'].notnull())].reset_index(drop=True)
    UK_Mean_Station_Pressure = UniversalKriging(
          input_df4['LON'],
          input_df4['LAT'],
          input_df4['Mean_Station_Pressure'],
          variogram_model="spherical",
          verbose=False,
          enable_plotting=False,
          drift_terms=["regional_linear"],
    )
    #Maximum_Sustained_Wind_Speed
    input_df5=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Maximum_Sustained_Wind_Speed'].notnull())].reset_index(drop=True)
    UK_Maximum_Sustained_Wind_Speed = UniversalKriging(
          input_df5['LON'],
          input_df5['LAT'],
          input_df5['Maximum_Sustained_Wind_Speed'],
          variogram_model="spherical",
          verbose=False,
          enable_plotting=False,
          drift_terms=["regional_linear"],
    )
    data_df=ver6_df[ver6_df['fih_date1']==day['date']]
    for j, row in data_df.iterrows():
      v1,s1=UK_Station_TDMEAN_c.execute('grid',row['LON'],row['LAT'])
      if station_data_df['Station_TDMEAN_c'].min()<=v1.data[0] and station_data_df['Station_TDMEAN_c'].max()>=v1.data[0]:
        ver6_df.at[j, 'Station_TDMEAN_c']=round(v1.data[0][0],1)

      v2,s2=UK_Mean_Wind_Speed.execute('grid',row['LON'],row['LAT'])
      if station_data_df['Mean_Wind_Speed'].min()<=v2.data[0] and station_data_df['Mean_Wind_Speed'].max()>=v2.data[0]:
        ver6_df.at[j, 'Mean_Wind_Speed']=round(v2.data[0][0],1)

      v3,s3=UK_Mean_Sea_Level_Pressure.execute('grid',row['LON'],row['LAT'])
      if station_data_df['Mean_Sea_Level_Pressure'].min()<=v3.data[0] and station_data_df['Mean_Sea_Level_Pressure'].max()>=v3.data[0]:
        ver6_df.at[j, 'Mean_Sea_Level_Pressure']=round(v3.data[0][0],1)

      v4,s4=UK_Mean_Station_Pressure.execute('grid',row['LON'],row['LAT'])
      if station_data_df['Mean_Station_Pressure'].min()<=v4.data[0] and station_data_df['Mean_Station_Pressure'].max()>=v4.data[0]:
        ver6_df.at[j, 'Mean_Station_Pressure']=round(v4.data[0][0],1)

      v5,s5=UK_Maximum_Sustained_Wind_Speed.execute('grid',row['LON'],row['LAT'])
      if station_data_df['Maximum_Sustained_Wind_Speed'].min()<=v5.data[0] and station_data_df['Maximum_Sustained_Wind_Speed'].max()>=v5.data[0]:
        ver6_df.at[j, 'Maximum_Sustained_Wind_Speed']=round(v5.data[0][0],1)
  except Exception as e:
    print(uniq_dates.loc[[i]])
    print("Kriging :"+str(e))
    print(traceback.format_exc())
print("Completed : 100.0%")

Completed : 0.0%
    index       date
20   1619 2001-09-26
Kriging :Singular matrix
Traceback (most recent call last):
  File "<ipython-input-57-40d5a319a855>", line 17, in <module>
    drift_terms=["regional_linear"],
  File "/usr/local/lib/python3.6/dist-packages/pykrige/uk.py", line 380, in __init__
    self.pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 842, in _find_statistics
    pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 777, in _krige
    res = np.linalg.solve(a, b)
  File "<__array_function__ internals>", line 6, in solve
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 394, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 88, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix

    index       date
21   1620 2001-10-01
Kri

/usr/local/lib/python3.6/dist-packages/pykrige/core.py:853: RuntimeWarning: invalid value encountered in sqrt
  sigma[i] = np.sqrt(ss)


Completed : 2.78%
     index       date
102   4674 2003-02-03
Kriging :Singular matrix
Traceback (most recent call last):
  File "<ipython-input-57-40d5a319a855>", line 17, in <module>
    drift_terms=["regional_linear"],
  File "/usr/local/lib/python3.6/dist-packages/pykrige/uk.py", line 380, in __init__
    self.pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 842, in _find_statistics
    pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 777, in _krige
    res = np.linalg.solve(a, b)
  File "<__array_function__ internals>", line 6, in solve
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 394, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 88, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix

     index       date
103   4675 2003-02-0

/usr/local/lib/python3.6/dist-packages/pykrige/core.py:853: RuntimeWarning: invalid value encountered in sqrt
  sigma[i] = np.sqrt(ss)


     index       date
128   5958 2003-10-16
Kriging :Singular matrix
Traceback (most recent call last):
  File "<ipython-input-57-40d5a319a855>", line 17, in <module>
    drift_terms=["regional_linear"],
  File "/usr/local/lib/python3.6/dist-packages/pykrige/uk.py", line 380, in __init__
    self.pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 842, in _find_statistics
    pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 777, in _krige
    res = np.linalg.solve(a, b)
  File "<__array_function__ internals>", line 6, in solve
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 394, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 88, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix



...

Completed : 13.89%
     index       date
507   9191 2006-03-01
Kriging :Singular matrix
Traceback (most recent call last):
  File "<ipython-input-57-40d5a319a855>", line 17, in <module>
    drift_terms=["regional_linear"],
  File "/usr/local/lib/python3.6/dist-packages/pykrige/uk.py", line 380, in __init__
    self.pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 842, in _find_statistics
    pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 777, in _krige
    res = np.linalg.solve(a, b)
  File "<__array_function__ internals>", line 6, in solve
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 394, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 88, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix

     index       date
509   9196 2006-03-

/usr/local/lib/python3.6/dist-packages/pykrige/core.py:853: RuntimeWarning: invalid value encountered in sqrt
  sigma[i] = np.sqrt(ss)


     index       date
538   9319 2006-04-20
Kriging :Singular matrix
Traceback (most recent call last):
  File "<ipython-input-57-40d5a319a855>", line 17, in <module>
    drift_terms=["regional_linear"],
  File "/usr/local/lib/python3.6/dist-packages/pykrige/uk.py", line 380, in __init__
    self.pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 842, in _find_statistics
    pseudo_inv,
  File "/usr/local/lib/python3.6/dist-packages/pykrige/core.py", line 777, in _krige
    res = np.linalg.solve(a, b)
  File "<__array_function__ internals>", line 6, in solve
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 394, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
  File "/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py", line 88, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix

     index       date
572   9686 2006-06-14
Kriging :Singula

/usr/local/lib/python3.6/dist-packages/pykrige/core.py:853: RuntimeWarning: invalid value encountered in sqrt
  sigma[i] = np.sqrt(ss)


Completed : 16.67%
Completed : 19.45%
Completed : 22.23%
Completed : 25.01%
Completed : 27.79%
Completed : 30.56%
Completed : 33.34%
Completed : 36.12%
Completed : 38.9%
Completed : 41.68%
Completed : 44.46%
Completed : 47.24%
Completed : 50.01%
Completed : 52.79%
Completed : 55.57%
Completed : 58.35%
Completed : 61.13%
Completed : 63.91%
Completed : 66.69%
Completed : 69.46%
Completed : 72.24%
Completed : 75.02%
Completed : 77.8%
Completed : 80.58%
Completed : 83.36%
Completed : 86.14%
Completed : 88.91%
Completed : 91.69%
Completed : 94.47%
Completed : 97.25%
Completed : 100.0%


In [67]:
ver6_df.shape

(34932, 43)

In [74]:
ver7_df=pd.DataFrame()
ver7_df[['IDATE','LAT','LON','ELEVATION_m','ACRES','PPT_mm','TMIN_c','TMAX_c','VPD9AM_hpa','VPD3PM_hpa','lfmc_mean','lfmc_stdv','NAMELSAD','population', \
    'Station_TDMEAN_c', 'Mean_Sea_Level_Pressure',	'Mean_Station_Pressure',	'Mean_Wind_Speed',	'Maximum_Sustained_Wind_Speed']]= \
ver6_df[['fih_date1','LAT','LON','ELEVATION','ACRES','rain_fall_mm','min_temperature_c','max_temperature_c','vp_9am_hpa','vp_3pm_hpa','lfmc_mean','lfmc_stdv','LGA','population',\
      'Station_TDMEAN_c', 'Mean_Sea_Level_Pressure',	'Mean_Station_Pressure',	'Mean_Wind_Speed',	'Maximum_Sustained_Wind_Speed']]
ver7_df['TMEAN_c']=ver6_df['max_temperature_c'].div(2)+ver6_df['min_temperature_c'].div(2)
ver7_df.shape

(34932, 20)

In [76]:
ver7_df=ver7_df.dropna()

In [77]:
ver7_df.shape

(33300, 20)

In [78]:
ver7_df.head()

,IDATE,LAT,LON,ELEVATION_m,ACRES,PPT_mm,TMIN_c,TMAX_c,VPD9AM_hpa,VPD3PM_hpa,lfmc_mean,lfmc_stdv,NAMELSAD,population,Station_TDMEAN_c,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Mean_Wind_Speed,Maximum_Sustained_Wind_Speed,TMEAN_c
0,2001-01-01,-31.3497,116.0366,206,850.9043,0.0,13.40,29.32,15.27,15.10,36.224998,18.932627,Chittering (S),2948,12.8,1011.7,992.4,17.3,43.9,21.360
1,2001-01-01,-32.1413,116.1956,266,18.3947,0.0,14.58,29.39,14.62,14.69,61.924999,16.209669,Armadale (C),52391,13.0,1011.8,993.0,14.8,43.9,21.985
2,2001-01-01,-32.0572,116.1932,312,0.4853,0.0,14.32,29.14,14.67,14.76,66.547501,13.480727,Kalamunda (C),48743,13.3,1011.7,992.9,15.2,43.9,21.730
3,2001-01-01,-32.0542,116.1940,311,5.3271,0.0,14.32,29.14,14.67,14.76,61.877499,13.119061,Kalamunda (C),48743,13.3,1011.7,992.9,15.2,43.9,21.730
4,2001-01-01,-32.0476,116.1925,309,0.5353,0.0,14.32,29.14,14.67,14.76,83.957497,14.985291,Kalamunda (C),48743,13.3,1011.7,992.9,15.2,43.9,21.730


In [79]:
ver7_df.to_excel('WesternAustraliaFinalModelInputDataSet.xlsx',index=False)
files.download("/content/WesternAustraliaFinalModelInputDataSet.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>